# Laboratorio de regresión - 4

SANTIAGO REYES CASTILLO  
12 DE FEBRERO, 2025  
745826

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

## LIBRERÍAS

In [19]:
# Librerías
from scipy import stats
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

## LEER LOS DATOS Y SEPARAR X Y Y

In [21]:
dataframe = pd.read_csv("Advertising.csv")

In [22]:
X = dataframe.iloc[:,1:-1]
y = dataframe.iloc[:,-1]

## SEPARAR TRAIN Y TEST

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.4, random_state=2)

## ESCALAR DATOS

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [27]:
X_std_train = scaler.transform(X_train)

In [28]:
X_test_std = scaler.transform(X_test)

## DEFINIR MODELOS

In [30]:
regression = LinearRegression()
ridge = Ridge(alpha=0.05)
lasso = Lasso(alpha=0.05)

regression.fit(X_std_train, y_train)
ridge.fit(X_std_train, y_train)
lasso.fit(X_std_train, y_train)

Lasso(alpha=0.05)

## CALCULAMOS LOS ERRORES, ESTADÍSTICOS t Y P-VALUES

In [36]:
def std_errors(X, y, model):

    y_pred = model.predict(X)

    RSS = np.sum((y_pred - y)**2)
    n, p = X.shape
    p = p + 1
   
    variance = RSS/(n-p)
    RSE = np.sqrt(RSS/(n-p))

    X_with_intercept = np.column_stack((np.ones(n), X)) 
    XT_X_inv = np.linalg.inv(X_with_intercept.T @ X_with_intercept)  
    std_errors = np.sqrt(np.diagonal(variance * XT_X_inv))
                     
                     
    return RSS, RSE, std_errors

In [38]:
def t_statistic(intercept, coef, standard_errors):
    
    t0 = intercept/(np.sqrt(standard_errors[0]))
    t1 = coef[0]/(np.sqrt(standard_errors[1]))
    t2 = coef[1]/(np.sqrt(standard_errors[2]))
    t3 = coef[2]/(np.sqrt(standard_errors[3]))

    return t0, t1, t2, t3

In [40]:
def p_values(y, t0, t1, t2, t3):

    p = 4  
    n = len(y_train)
    p_b0 = 2*(1 - stats.t.cdf(np.abs(t0), n-p))
    p_b1 = 2*(1 - stats.t.cdf(np.abs(t1), n-p))
    p_b2 = 2*(1 - stats.t.cdf(np.abs(t2), n-p))
    p_b3 = 2*(1 - stats.t.cdf(np.abs(t3), n-p))

    return p_b0, p_b1, p_b2, p_b3

In [42]:
RSS_lr, RSE_lr, std_errs_lr = std_errors(X_test_std, y_test, regression)
t0, t1, t2, t3 = t_statistic(regression.intercept_, regression.coef_, std_errs_lr)
p_values_lr =p_values(y_test, t0, t1, t2, t3)

In [48]:
RSS_ridge, RSS_ridge, std_errs_ridge = std_errors(X_test_std, y_test, ridge)
t0, t1, t2, t3 = t_statistic(ridge.intercept_, ridge.coef_, std_errs_ridge)
p_values_ridge = p_values(y_test, t0, t1, t2, t3)

In [50]:
RSS_lasso, RSS_ridge_lasso, std_errs_lasso = std_errors(X_test_std, y_test, lasso)
t0, t1, t2, t3 = t_statistic(lasso.intercept_, lasso.coef_, std_errs_lasso)
p_values_lasso = p_values(y_test, t0, t1, t2, t3)

## PARÁMETROS, R2 Y P-VALUES

In [52]:
y_pred_lr = regression.predict(X_test_std)
r2_lr = r2_score(y_test, y_pred_lr)

y_pred_ridge = regression.predict(X_test_std)
r2_ridge = r2_score(y_test, y_pred_ridge)

y_pred_lasso = regression.predict(X_test_std)
r2_lasso = r2_score(y_test, y_pred_lasso)


In [54]:
results = {
    'TIPO DE MODELO': ['REGRESION', 'RIDGE', 'LASSO'],
    'R2': [r2_lr, r2_ridge, r2_lasso],
    'INTERCEPTO': [regression.intercept_, ridge.intercept_, lasso.intercept_ ],
    'P-VALUE B0': [p_values_lr[0], p_values_ridge[0], p_values_lasso[0]],
    'TV': [regression.coef_[0], ridge.coef_[0], lasso.coef_[0]],
    'P-VALUE B1': [p_values_lr[1], p_values_ridge[1], p_values_lasso[1]],
    'RADIO': [regression.coef_[1], ridge.coef_[1], lasso.coef_[1]],
    'P-VALUE B2': [p_values_lr[2], p_values_ridge[2], p_values_lasso[2]],
    'NEWSPAPER': [regression.coef_[2], ridge.coef_[2], lasso.coef_[2]],
    'P-VALUE B3': [p_values_lr[3], p_values_ridge[3], p_values_lasso[3]]
}


In [56]:
results = pd.DataFrame(results)
results

,TIPO DE MODELO,R2,INTERCEPTO,P-VALUE B0,TV,P-VALUE B1,RADIO,P-VALUE B2,NEWSPAPER,P-VALUE B3
0,REGRESION,0.866837,13.78875,0.0,3.896139,8.437695e-15,2.707447,2.787402e-08,-0.004956,0.990546
1,RIDGE,0.866837,13.78875,0.0,3.893713,8.437695e-15,2.705840,2.826211e-08,-0.003832,0.992689
2,LASSO,0.866837,13.78875,0.0,3.852807,1.310063e-14,2.663730,4.188021e-08,0.000000,1.000000


## CONCLUSIONES 

Podemos observar, que los R2 de los 3 modelos es prácticamente el mismo, sin embargo, los coeficientes de cada uno varían entre los modelos. Lo más destacable es que en el caso de la penalización con Lasso, penaliza tanto la predictora de Newspaper que la convierte en 0 para el modeo, mientras que Ridge no.  
Además, los pvalues de los predictores para TV y Radio son prácticamente 0 por lo que no hay coincidencia en que predigan correctamente la variable de salida, pero del otro lado, el pvalue del Newspaper es muy cercano a 1, indicando lo contrario.  
Finalmente, podemos ver que utilizar las normas L1 y L2 nos pueden ayudar a predecir de una manera un poco más meticulosa en una regresión lineal, sin embargo, los escenarios en que sean significativamente mejores a la regresión.  
La principal área de mejora de este trabajo es encontrar los valores óptimos para el lambda de las penalizaciones, ya que en este caso se eligió arbitrariamente y existen maneras de optimizarlos. 